# Factory Planning II

## Objective and Prerequisites

Are you up for a major production planning challenge? Try this example where you will learn how to create an optimal production plan that will not only maximize profits, but also determine which month in which to perform maintenance operations on your machines.

More information on this type of model can be found in example #4 of the fifth edition of Modeling Building in Mathematical Programming by H. P. Williams on pages 256 and 302 – 303.


---
## Problem Description

A factory makes seven products (Prod 1 to Prod 7) using a range of machines including:

- Four grinders
- Two vertical drills
- Three horizontal drills
- One borer
- One planer

Each product has a defined profit contribution per unit sold (defined as the sales price per unit minus the cost of raw materials). In addition, the manufacturing of each product requires a certain amount of time on each machine (in hours). The profit and manufacturing time value are shown below. A dash indicates that the manufacturing process for the given product does not require that machine.

| <i></i> | PROD1 | PROD2 | PROD3 | PROD4 | PROD5 | PROD6 | PROD7 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Profit | 10 | 6 | 8 | 4 | 11 | 9 | 3 |
| Grinding | 0.5 | 0.7 | - | - | 0.3 | 0.2 | 0.5 |
| Vertical Drilling | 0.1 | 0.2 | - | 0.3 | - | 0.6 | - |
| Horizontal Drilling | 0.2 | - | 0.8 | - | - | - | 0.6 |
| Boring | 0.05 | 0.03 | - | 0.07 | 0.1 | - | 0.08 |
| Planning | - | - | 0.01 | - | 0.05 | - | 0.05 |

Instead of pre-defining a maintenance schedule for the machines, as was done in Factory Planning I, in this version of the model we will also optimize the maintenance schedule.

The maintenance requirements are as follows:

- Each machine must be down for maintenance in one month of the six.
- The exception to the above are the grinding machines as only two of them need to be down during the six months.

There are limitations on how many of each product can be sold in a given month. These limits are shown below:

| Month | PROD1 | PROD2 | PROD3 | PROD4 | PROD5 | PROD6 | PROD7 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| January | 500 | 1000 | 300 | 300 | 800 | 200 | 100 |
| February | 600 | 500 | 200 | 0 | 400 | 300 | 150 |
| March | 300 | 600 | 0 | 0 | 500 | 400 | 100 |
| April | 200 | 300 | 400 | 500 | 200 | 0 | 100 |
| May | 0 | 100 | 500 | 100 | 1000 | 300 | 0 |
| June | 500 | 500 | 100 | 300 | 1100 | 500 | 60 |

Up to 100 units of each product may be stored in inventory at a cost of $0.50 per unit per month. At the start of January, there is no product inventory. However, by the end of June, there should be 50 units of each product in inventory.

The factory produces products six days a week using two eight-hour shifts per day. It may be assumed that each month consists of 24 working days. Also, for the purposes of this model, there are no production sequencing issues that need to be taken into account.

What should the production and maintenance plans look like? Also, is it possible to recommend any price increases and determine the value of acquiring any new machines?

This problem is based on a larger model built for the Cornish engineering company of Holman Brothers.

---
## Model Formulation

### Sets and Indices

$t \in \text{Months}=\{\text{Jan},\text{Feb},\text{Mar},\text{Apr},\text{May},\text{Jun}\}$: Set of months.

$p \in \text{Products}=\{1,2,\dots,7\}$: Set of products.

$m \in \text{Machines}=\{\text{Grinder},\text{VertDrill},\text{horiDrill},\text{Borer},\text{Planer}\}$: Set of machines.

### Parameters

$\text{hours\_per\_month} \in \mathbb{R}^+$: Time (in hours/month) available at any machine on a monthly basis. It results from multiplying the number of working days (24 days) by the number of shifts per day (2) by the duration of a shift (8 hours).

$\text{max\_inventory} \in \mathbb{N}$: Maximum number of units of a single product type that can be stored in inventory at any given month.

$\text{holding\_cost} \in \mathbb{R}^+$: Monthly cost (in USD/unit/month) of keeping in inventory a unit of any product type.

$\text{store\_target} \in \mathbb{N}$: Number of units of each product type to keep in inventory at the end of the planning horizon.

$\text{profit}_{p} \in \mathbb{R}^+$: Profit (in USD/unit) of product $p$.

$\text{installed}_{m} \in \mathbb{N}$: Number of machines of type $m$ installed in the factory.

$\text{down\_req}_{m} \in \mathbb{N}$: Number of machines of type $m$ that should be scheduled for maintenance at some point in the planning horizon.

$\text{time\_req}_{m,p} \in \mathbb{R}^+$: Time (in hours/unit) needed on machine $m$ to manufacture one unit of product $p$.

$\text{max\_sales}_{t,p} \in \mathbb{N}$: Maximum number of units of product $p$ that can be sold at month $t$.

### Decision Variables

$\text{make}_{t,p} \in \mathbb{R}^+$: Number of units of product $p$ to manufacture at month $t$.

$\text{store}_{t,p} \in [0, \text{max\_inventory}] \subset \mathbb{R}^+$: Number of units of product $p$ to store at month $t$.

$\text{sell}_{t,p} \in [0, \text{max\_sales}_{t,p}] \subset \mathbb{R}^+$: Number of units of product $p$ to sell at month $t$.

$\text{repair}_{t,m} \in \{0,1,\dots, \text{down\_req}_{m}\} \subset \mathbb{N}$: Number of machines of type $m$ scheduled for maintenance at month $t$.

**Assumption:** We can produce fractional units.

### Objective Function

- **Profit:** Maximize the total profit (in USD) of the planning horizon.

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Months}}\sum_{p \in \text{Products}}
(\text{profit}_{p} \cdot \text{make}_{t,p} - \text{holding\_cost} \cdot \text{store}_{t,p})
\tag{0}
\end{equation}

### Constraints

- **Initial Balance:** For each product $p$, the number of units produced should be equal to the number of units sold plus the number stored (in units of product).

\begin{equation}
\text{make}_{\text{Jan},p} = \text{sell}_{\text{Jan},p} + \text{store}_{\text{Jan},p} \quad \forall p \in \text{Products}
\tag{1}
\end{equation}

- **Balance:** For each product $p$, the number of units produced in month $t$ and previously stored should be equal to the number of units sold and stored in that month (in units of product).

\begin{equation}
\text{store}_{t-1,p} + \text{make}_{t,p} = \text{sell}_{t,p} + \text{store}_{t,p} \quad \forall (t,p) \in \text{Months} \setminus \{\text{Jan}\} \times \text{Products}
\tag{2}
\end{equation}

- **Inventory Target:** The number of units of product $p$ kept in inventory at the end of the planning horizon should hit the target (in units of product).

\begin{equation}
\text{store}_{\text{Jun},p} = \text{store\_target} \quad \forall p \in \text{Products}
\tag{3}
\end{equation}

- **Machine Capacity:** Total time used to manufacture any product at machine type $m$ cannot exceed its monthly capacity (in hours).

\begin{equation}
\sum_{p \in \text{Products}}\text{time\_req}_{m,p}*\text{make}_{t,p} \leq \text{hours\_per\_month}*(\text{installed}_m - \text{repair}_{t,m}) \quad \forall (t,m) \in \text{Months} \times \text{Machines}
\tag{4}
\end{equation}

- **Maintenance**: The number of machines of type $m$ scheduled for maintenance should meet the requirement.

\begin{equation}
\sum_{t \in \text{Months}}\text{repair}_{t,m} = \text{down\_req}_m \quad \forall m \in \text{Machines}
\tag{5}
\end{equation}

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [74]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo

## Input Data
We define all the input data of the model.

In [75]:
# Parameters

products = ["Prod1", "Prod2", "Prod3", "Prod4", "Prod5", "Prod6", "Prod7"]
machines = ["grinder", "vertDrill", "horiDrill", "borer", "planer"]
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

profit = {
    "Prod1": 10,
    "Prod2": 6,
    "Prod3": 8,
    "Prod4": 4,
    "Prod5": 11,
    "Prod6": 9,
    "Prod7": 3,
}

time_req = {
    "grinder": {"Prod1": 0.5, "Prod2": 0.7, "Prod5": 0.3, "Prod6": 0.2, "Prod7": 0.5},
    "vertDrill": {"Prod1": 0.1, "Prod2": 0.2, "Prod4": 0.3, "Prod6": 0.6},
    "horiDrill": {"Prod1": 0.2, "Prod3": 0.8, "Prod7": 0.6},
    "borer": {"Prod1": 0.05, "Prod2": 0.03, "Prod4": 0.07, "Prod5": 0.1, "Prod7": 0.08},
    "planer": {"Prod3": 0.01, "Prod5": 0.05, "Prod7": 0.05},
}

# number of each machine available
installed = {"grinder": 4, "vertDrill": 2, "horiDrill": 3, "borer": 1, "planer": 1}

# number of machines that need to be under maintenance
down_req = {"grinder": 2, "vertDrill": 2, "horiDrill": 3, "borer": 1, "planer": 1}

# market limitation of sells
max_sales = {
    ("Jan", "Prod1"): 500,
    ("Jan", "Prod2"): 1000,
    ("Jan", "Prod3"): 300,
    ("Jan", "Prod4"): 300,
    ("Jan", "Prod5"): 800,
    ("Jan", "Prod6"): 200,
    ("Jan", "Prod7"): 100,
    ("Feb", "Prod1"): 600,
    ("Feb", "Prod2"): 500,
    ("Feb", "Prod3"): 200,
    ("Feb", "Prod4"): 0,
    ("Feb", "Prod5"): 400,
    ("Feb", "Prod6"): 300,
    ("Feb", "Prod7"): 150,
    ("Mar", "Prod1"): 300,
    ("Mar", "Prod2"): 600,
    ("Mar", "Prod3"): 0,
    ("Mar", "Prod4"): 0,
    ("Mar", "Prod5"): 500,
    ("Mar", "Prod6"): 400,
    ("Mar", "Prod7"): 100,
    ("Apr", "Prod1"): 200,
    ("Apr", "Prod2"): 300,
    ("Apr", "Prod3"): 400,
    ("Apr", "Prod4"): 500,
    ("Apr", "Prod5"): 200,
    ("Apr", "Prod6"): 0,
    ("Apr", "Prod7"): 100,
    ("May", "Prod1"): 0,
    ("May", "Prod2"): 100,
    ("May", "Prod3"): 500,
    ("May", "Prod4"): 100,
    ("May", "Prod5"): 1000,
    ("May", "Prod6"): 300,
    ("May", "Prod7"): 0,
    ("Jun", "Prod1"): 500,
    ("Jun", "Prod2"): 500,
    ("Jun", "Prod3"): 100,
    ("Jun", "Prod4"): 300,
    ("Jun", "Prod5"): 1100,
    ("Jun", "Prod6"): 500,
    ("Jun", "Prod7"): 60,
}

holding_cost = 0.5
max_inventory = 100
store_target = 50
hours_per_month = 2 * 8 * 24

## Model Deployment
We create a model and the variables. We set the UpdateMode parameter to 1 (which simplifies the code – see the documentation for more details). For each product (seven kinds of products) and each time period (month), we will create variables for the amount of which products will get manufactured, held, and sold. In each month, there is an upper limit on the amount of each product that can be sold. This is due to market limitations. For each type of machine and each month we create a variable d, which tells us how many machines are down in this month of this type.

In [76]:
m = pyo.ConcreteModel("Factory Planning II")

m.make = pyo.Var(
    months, products, domain=pyo.NonNegativeReals, name="Make"
)  # quantity manufactured
m.store = pyo.Var(
    months, products, bounds=(0, max_inventory), name="Store"
)  # quantity stored
m.sell = pyo.Var(
    months, products, bounds=lambda m, mo, p: (0, max_sales[(mo, p)]), name="Sell"
)  # quantity sold

m.repair = pyo.Var(
    months,
    machines,
    domain=pyo.Integers,
    bounds=lambda m, mo, ma: (0, down_req[ma]),
    name="Repair",
)  # number of machines down

Next, we insert the constraints.
The balance constraints ensure that the amount of product that is in the storage in the prior month and the amount that gets manufactured equals the amount that is sold and held for each product in the current month. This ensures that all products in the model are manufactured in some month. The initial storage is empty.

In [77]:
# 1. Initial Balance
def balance_0(m, p):
    return m.make[months[0], p] == m.sell[months[0], p] + m.store[months[0], p]


m.balance_0 = pyo.Constraint(products, rule=balance_0, name="initial balance")


# 2. Balance
def balance(m, p, mo):
    return (
        m.store[months[months.index(mo) - 1], p] + m.make[mo, p]
        == m.sell[mo, p] + m.store[mo, p]
    )


m.balance = pyo.Constraint(
    products, [m for m in months if m != months[0]], rule=balance, name="balance"
)

The endstore constraints force that at the end of the last month the storage contains the specified amount of each product.

In [78]:
# 3. Inventory Target
m.inventory_balance = pyo.Constraint(
    products, rule=lambda m, p: m.store[months[-1], p] == store_target
)

The capacity constraints ensure that for each month the time all products require on a certain kind of machine is lower or equal than the available hours for that machine in that month multiplied by the number of available machines in that month. Each product requires some machine hours on different machines. Each machine is down in one or more months due to maintenance, so the number and types of available machines varies per month. There can be multiple machines per machine type.

In [79]:
# 4. Machine Capacity
def machine_capacity(m, ma, mo):
    return sum(
        time_req[ma][p] * m.make[mo, p] for p in time_req[ma]
    ) <= hours_per_month * (installed[ma] - m.repair[mo, ma])


m.machine_capacity = pyo.Constraint(machines, months, rule=machine_capacity)

The maintenance constraints ensure that the specified number and types of machines are down due maintenance in some month. Which month a machine is down is now part of the optimization.

In [80]:
# 5. Maintenance
def maintenance(m, ma):
    return sum(m.repair[mo, ma] for mo in months) == down_req[ma]


m.maintenance = pyo.Constraint(machines, rule=maintenance)

The objective is to maximize the profit of the company, which consists of the profit for each product minus cost for storing the unsold products. This can be stated as:

In [81]:
# 0. Objective Function
obj = lambda m: sum(
    profit[product] * m.sell[month, product] - holding_cost * m.store[month, product]
    for month in months
    for product in products
)
m.obj = pyo.Objective(rule=obj, sense=pyo.maximize)

Next, we start the optimization and Gurobi finds the optimal solution.

In [82]:
# solve
pyo.SolverFactory("cbc").solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 108855.0
  Upper bound: 108855.0
  Number of objectives: 1
  Number of constraints: 67
  Number of variables: 133
  Number of binary variables: 12
  Number of integer variables: 30
  Number of nonzeros: 71
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.03
  Wallclock time: 0.04
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch an

---
## Analysis

The result of the optimization model shows that the maximum profit we can achieve is $\$108,855.00$. This is an increase of $\$15,139.82$ over the course of six months compared to the Factory Planning I example as a result of being able to pick the maintenance schedule as opposed to having a fixed one. Let's see the solution that achieves that optimal result. 

### Production Plan
This plan determines the amount of each product to make at each period of the planning horizon. For example, in February we make 600 units of product Prod1.

In [83]:
rows = months.copy()
columns = products.copy()
make_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in m.make.keys():
    if abs(m.make[month, product].value) > 1e-6:
        make_plan.loc[month, product] = np.round(m.make[month, product].value, 1)
make_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,500.0,1000.0,300.0,300.0,800.0,200.0,100.0
Feb,600.0,500.0,200.0,0.0,400.0,300.0,150.0
Mar,400.0,700.0,100.0,100.0,600.0,400.0,200.0
Apr,0.0,0.0,0.0,0.0,0.0,0.0,0.0
May,0.0,100.0,500.0,100.0,1000.0,300.0,0.0
Jun,550.0,550.0,150.0,350.0,1150.0,550.0,110.0


### Sales Plan
This plan defines the amount of each product to sell at each period of the planning horizon. For example, in February we sell 600 units of product Prod1.

In [84]:
rows = months.copy()
columns = products.copy()
sell_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in m.sell.keys():
    if abs(m.sell[month, product].value) > 1e-6:
        sell_plan.loc[month, product] = np.round(m.sell[month, product].value, 1)
sell_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,500.0,1000.0,300.0,300.0,800.0,200.0,100.0
Feb,600.0,500.0,200.0,0.0,400.0,300.0,150.0
Mar,300.0,600.0,0.0,0.0,500.0,400.0,100.0
Apr,100.0,100.0,100.0,100.0,100.0,0.0,100.0
May,0.0,100.0,500.0,100.0,1000.0,300.0,0.0
Jun,500.0,500.0,100.0,300.0,1100.0,500.0,60.0


### Inventory Plan
This plan reflects the amount of product in inventory at the end of each period of the planning horizon. For example, at the end of February we have zero units of Prod1 in inventory.

In [85]:
rows = months.copy()
columns = products.copy()
store_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in m.store.keys():
    if abs(m.store[month, product].value) > 1e-6:
        store_plan.loc[month, product] = np.round(m.store[month, product].value, 1)
store_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Feb,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mar,100.0,100.0,100.0,100.0,100.0,0.0,100.0
Apr,0.0,0.0,0.0,0.0,0.0,0.0,0.0
May,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jun,50.0,50.0,50.0,50.0,50.0,50.0,50.0


### Maintenance Plan
This plan shows the maintenance plan for each period of the planning horizon. For example, 2 machines of type grinder will be down for maintenance in April.

In [86]:
rows = months.copy()
columns = machines.copy()
repair_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, machine in m.repair.keys():
    if abs(m.repair[month, machine].value) > 1e-6:
        repair_plan.loc[month, machine] = m.repair[month, machine].value
repair_plan

,grinder,vertDrill,horiDrill,borer,planer
Jan,0.0,0.0,0.0,0.0,0.0
Feb,0.0,0.0,0.0,0.0,0.0
Mar,0.0,0.0,0.0,0.0,0.0
Apr,0.0,1.0,1.0,1.0,1.0
May,2.0,1.0,0.0,0.0,0.0
Jun,0.0,0.0,2.0,0.0,0.0


## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.